In [1]:
import torch
import torch.nn as nn
from IPython.display import clear_output
from IPython.core.debugger import set_trace
from torch.nn import functional as F
from torch import optim
import math

%load_ext autoreload
%autoreload 2
import dlc_practical_prologue as dl

In [17]:
train_input, train_target, test_input, test_target = dl.load_data(flatten=False)
train_input = torch.functional.F.avg_pool2d(train_input, kernel_size = 2)
test_input = torch.functional.F.avg_pool2d(test_input, kernel_size = 2)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [18]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.block1 = nn.Sequential(nn.Conv2d(1,32, kernel_size=5,stride=1),
                                    nn.MaxPool2d(kernel_size=2,stride=2),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU())
        self.block2 = nn.Sequential(nn.Conv2d(32,16, kernel_size=5,stride=1),
                                    nn.MaxPool2d(kernel_size=2,stride=2),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU())
        self.lins = nn.Sequential(nn.Linear(256, 84),
                                  nn.ReLU(),
                                  nn.Linear(84, 10))

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.lins(x.view(-1,256))
        return x

In [24]:
class Net2(nn.Module):
    def __init__(self,n_hidden = 100,chan = 1):
        super(Net2,self).__init__()
        self.hidden = n_hidden
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(chan,32,kernel_size=3),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3),
            nn.MaxPool2d(kernel_size=2,stride=2)
            ,nn.BatchNorm2d(64)
        )
        self.classifier = nn.Sequential(
            nn.Linear(2,n_hidden),
           # nn.Dropout(0.5),
            nn.Linear(n_hidden,10)
            #nn.Softmax2d()
        )
    def forward(self,x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.classifier(x.view(x.size(0),-1))
        return x

In [20]:
def train_model(model,train_input,train_target,nb_epochs=5):
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    batch_size = 100
    criterion = nn.CrossEntropyLoss()
    for e in range(nb_epochs):
        clear_output(wait=True)
        print("Progression:{:.2f}".format(e/nb_epochs*100))
        for inputs,targets in zip(train_input.split(batch_size),
                            train_target.split(batch_size)):
            output = model(inputs)
            loss = criterion(output,targets)
            model.zero_grad()
            loss.backward()
            optimizer.step()

In [21]:
def get_accuracy(model,inputs,targets):
    assert(inputs.size(0) == targets.size(0))
    tot_loss = 0
    nb_correct = 0
    batch_size = 20
    for train,target in zip(inputs.split(batch_size),
                           targets.split(batch_size)):
        pred = model(train)
        pred = torch.argmax(pred,axis = 1)
        nb_correct += (pred == target).int().sum().item()
    accuracy = nb_correct /inputs.size(0)
    print("accuracy: %.2f" % (accuracy) )
    return accuracy

In [22]:
def Kfold_CV(classtype,inputs,targets,K=4):
    assert(K>=2)
    N = inputs.size(0)
    indxes = torch.randperm(N)\
                  .split(int(N/K))
    accs = torch.empty(K)
    for k in range(K):
        model = classtype()
        
        test_indx = indxes[k]
        train_indx = torch.cat((indxes[:k]+indxes[k+1:]),0)
        
        train_inp,train_targ = inputs[train_indx],targets[train_indx]
        test_inp,test_targ = inputs[test_indx],targets[test_indx]
        train_model(model,train_inp,train_targ)
        acc = get_accuracy(model,train_inp,train_targ)
        accs[k] = acc
    print("Accuracies for {}-fold:{}".format(K,accs.tolist()))
    print("Mean acc:{}".format(accs.mean()))

In [25]:
model = Net2()
train_model(model,train_input,train_target)
get_accuracy(model,train_input,train_target)

Progression:80.00
accuracy: 0.99


0.995

In [26]:
Kfold_CV(Net2,train_input,train_target)

Progression:80.00
accuracy: 0.99
Accuracies for 4-fold:[0.9866666793823242, 0.9879999756813049, 0.9906666874885559, 0.9893333315849304]
Mean acc:0.9886666536331177


## Dealing with double Images

In [177]:
a, b, c, d = dl.load_data(flatten=False)
train2_input, train2_target, train2_classes = dl.mnist_to_pairs(28,a,b)
test2_input, test2_target, test2_classes = dl.mnist_to_pairs(28,a,b)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [178]:
test2_classes[:,0]

tensor([4, 3, 2, 0, 7, 9, 3, 1, 0, 3, 4, 8, 2, 8, 0, 8, 8, 3, 9, 4, 7, 7, 9, 9,
        7, 3, 6, 0])

In [179]:
torch.cat((torch.rand(3,2),torch.rand(6,2)),dim=0)

tensor([[0.9574, 0.8164],
        [0.6936, 0.4115],
        [0.3574, 0.7729],
        [0.8452, 0.5438],
        [0.6778, 0.6991],
        [0.8857, 0.2749],
        [0.7880, 0.4802],
        [0.1622, 0.6052],
        [0.1941, 0.2088]])

In [185]:
class LeNet2imgs(nn.Module):
    def __init__(self,chan = 1):
        super(LeNet2imgs,self).__init__()
        self.net0 = Net2()
        self.net1 = Net2()
        self.linblock = nn.Sequential(nn.Linear(20,2),
                                      nn.ReLU())
        
    def forward(self,x):
        x0 = self.net0(x[:,0].unsqueeze(1))
        x1 = self.net1(x[:,1].unsqueeze(1))
        comp = torch.cat((x0,x1),dim=1)
        comp = self.linblock(comp)
        return x0,x1,comp

In [186]:
net2 = LeNet2imgs()

In [187]:
temp = train2_input[:3]

In [188]:
net2(train2_input)

(tensor([[-6.3500e-01,  2.1712e-01,  1.9979e-01,  6.4478e-01,  9.3705e-02,
          -1.4389e-01,  3.3843e-01,  1.8287e-01, -6.8561e-02,  1.0964e-01],
         [-7.4665e-01,  1.5814e-01,  3.7123e-02,  5.8379e-01,  2.8070e-01,
          -2.6475e-01,  4.1695e-02,  4.3801e-01, -8.3131e-03,  5.5023e-01],
         [-2.4485e-01,  5.2345e-01, -6.8252e-03,  2.6208e-01, -5.2334e-01,
          -1.1947e-01,  2.0169e-01, -1.1570e-01,  1.1686e-01,  3.4315e-01],
         [-3.5936e-01,  1.5088e-01,  4.9007e-02,  5.6661e-01, -8.4083e-01,
           4.1519e-02, -5.7859e-01, -4.2253e-02, -1.5741e-01,  4.2106e-01],
         [-2.3804e-01,  2.1723e-01, -6.9425e-02,  5.1032e-01,  2.5273e-01,
           2.8984e-01,  1.5738e-01,  3.1906e-02,  1.9539e-01, -2.3209e-01],
         [-2.0402e-01,  3.7970e-02,  2.8363e-02,  3.6457e-01,  3.4531e-01,
           1.8474e-01,  2.6056e-01, -6.4407e-02, -2.9155e-01, -4.8341e-01],
         [ 1.6206e-01,  5.9672e-01,  3.9365e-01,  6.2981e-01, -1.5756e-01,
          -3.6091e-

In [193]:
def train_double_model(model,train_input,train_target,train_classes,nb_epochs=5):
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    batch_size = 100
    crit0 = nn.CrossEntropyLoss()
    crit1 = nn.CrossEntropyLoss()
    crit_comp = nn.CrossEntropyLoss()
    for e in range(nb_epochs):
        clear_output(wait=True)
        print("Progression:{:.2f}".format(e/nb_epochs*100))
        for inputs,comp_targs,classes in zip(train_input.split(batch_size),
                                           train_target.split(batch_size),
                                           train_classes.split(batch_size)):
            targ0 = classes[:,0]
            targ1 = classes[:,1]
            x0,x1,comp = model(inputs)
            loss0 = crit0(x0,targ0)
            loss1 = crit1(x1,targ1)
            loss_class = crit_comp(comp,comp_targs)
            totloss = loss0 + loss1 + loss_class
            model.zero_grad()
            totloss.backward()
            optimizer.step()

In [204]:
def accuracy_double_model(model,train_input,train_target,train_classes):
    assert(train_input.size(0) == train_target.size(0))
    N = train_input.size(0)
    tot_loss = 0
    nb_correct = 0
    batch_size = 20
    
    #given a prediction powre and the target, output the number of correctly classified samples
    add_res = lambda pred,target:(torch.argmax(pred,axis = 1) == target).int().sum().item()
    
    score0 = 0
    score1 = 0
    scorecomp = 0
    
    for inputs,comp_targs,classes in zip(train_input.split(batch_size),
                                           train_target.split(batch_size),
                                        train_classes.split(batch_size)):
        targ0 = classes[:,0]
        targ1 = classes[:,1]
        x0,x1,comp = model(inputs)
        
        score0 += add_res(x0,targ0)
        score1 += add_res(x1,targ1)
        
        scorecomp += add_res
        
    acc0 = score0 / N
    acc1 = score1 / N
    acc_comp = scorecomp / N
    
    print("Accuracy first Network: %.2f" % (acc0) )
    print("Accuracy second Network: %.2f" % (acc1))
    print("Accuracy comparison: %.2f" % (acc_comp))

    return acc0,acc1,acc_comp

In [208]:
train_double_model(net2,train2_input,train2_target,train2_classes)
accuracy_double_model(net2,train2_input,train2_target,train2_classes)

Progression:80.00


RuntimeError: The size of tensor a (2) must match the size of tensor b (20) at non-singleton dimension 1

In [210]:
%debug

> /home/ymentha/anaconda3/envs/ML/lib/python3.7/site-packages/torch/tensor.py(28)wrapped()
     26     def wrapped(*args, **kwargs):
     27         try:
---> 28             return f(*args, **kwargs)
     29         except TypeError:
     30             return NotImplemented



ipdb>  u


> <ipython-input-204-866b3624b637>(24)accuracy_double_model()
     22         score0 += add_res(x0,targ0)
     23         score1 += add_res(x1,targ1)
---> 24         scorecomp += (comp == comp_targs).int().sum().item()
     25 
     26     acc0 = score0 / N



ipdb>  comp.shape


torch.Size([20, 2])


ipdb>  comp_targs.shape


torch.Size([20])


ipdb>  q
